1. 사용자 입력을 음성으로 받는다.
2. STT를 적용하여 텍스트로 변환한다.
3. 변환된 텍스트를 입력으로 하여 프롬프트 엔지니어링을 해 api 요청을 보낸다.
4. 반환받은 응답을 TTS를 적용하여 음성으로 재생한다.
(+) 2에서 입력된 텍스트와 4에서 반환된 응답을 채팅 내역 보듯이 (카카오톡 대화처럼) 현출되도록 출력한다

[1단계 : 토론 설정]
1. 봇이 먼저 질문하기 (음성으로) : 토론하고 싶은 주제와 토론하고 싶은 상대 MBTI를 말해주세요 예) "MBTI 토론 파트너입니다! 토론하고 싶은 주제와 상대할 MBTI를 말씀해주세요"
2. 사용자가 주제와 MBTI 말하기 (음성으로) : 예) "계획 여행과 즉흥 여행을 주제로 ENTP와 토론하고 싶어"
3. 사용자의 음성이 STT를 통해 텍스트로 변환됨 -> '주제'와 'MBTI'를 추출
4. AI에서 system prompt 설정 : {MBTI} {주제} 어떤 규칙을 지켜야 하는지

[2단계 : 토론 진행]
1. 봇이 토론 시작을 알림 : '여행'을 주제로 ENTP와 토론을 시작하겠습니다. 당신의 의견을 먼저 말씀해주세요.
2. 음성으로 내 의견을 말함
3. 봇이 설정된 페르소나로 응답을 받으면 TTS로 다시 음성으로 말함

---

In [11]:
#!pip install playsound
#!pip install pygame

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

### 텍스트 -> 음성 변환 함수 (TTS)

In [13]:
# 텍스트를 음성으로 변환하고 재생하는 함수
from gtts import gTTS
import uuid
import pygame

pygame.mixer.init()

def speak(text):
    print(f'🤖 : {text}')
    tts = gTTS(text=text, lang='ko')            # gTTS로 텍스트를 음성으로 변환
    filename = f'speech_{uuid.uuid4()}.mp3'     # mp3 파일로 저장
    tts.save(filename) 
    pygame.mixer.init()                         
    
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(60)

    pygame.mixer.music.unload()
    os.remove(filename)

### 음성 -> 텍스트 변환 함수 (STT)

In [14]:
# 사용자의 음성을 텍스트로 변환하는 함수
import speech_recognition as sr

recognizer = sr.Recognizer()

recognizer.pause_threshold = 1.5

def listen():
    with sr.Microphone() as source:
        print('🤖듣는 중....')
        #recognizer.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = recognizer.listen(source)
            txt = recognizer.recognize_google(audio, language='ko-KR')
            print(f'👤사용자 : {txt}')
            return txt
        except sr.UnknownValueError:
            speak('알아듣지 못했어요 다시 말씀해주세요')
            return None

### 텍스트에서 주제와 MBTI 추출 함수

In [15]:
# 텍스트에서 주제와 MBTI를 추출하는 함수
from openai import OpenAI
import json

def extract_debate_info(user_text):
    client = OpenAI()
    if not user_text:
        return None
    
    system_instruction="""
    너는 사용자의 문장에서 '토론 주제'와 '토론하고 싶은 상대 MBTI'를 추출하는 고도의 언어 분석 전문가야
    아래에 주어진 규칙에 따라 정보를 추출하고 JSON 형식으로만 응답해줘

    ## 규칙 ##
    1. 응답은 반드시 {'topic' : '추출된 토론 주제','mbti' : '추출된 MBTI 유형'} 형식의 JSON 객체여야 해
    2. MBTI는 반드시 4글자 영문 대문자로 추출해줘
    3. 만약 특정 정보를 찾을 수 없다면 해당 값은 추출 실패로 지정해줘
    4. 토론 주제는 문장에서 핵심적인 내용만 간결하게 요약하여 추출해줘
    5. 'A vs B'와 같이 두 가지 선택지 중 하나를 고르는 밸런스 게임 주제를 정확하게 인식해야 해. 'vs'를 '대'나 '돼'등으로 발음해도 문맥을 파악하여 'A vs B' 전체를 하나의 주제로 추출해야 해

    ## 규칙 5번에 대한 예시 ##
    - 입력 : 10분 장기자랑 하기 돼 친한 친구 10년 못 보기 entp
    - 출력 : `{'topic': '10분 장기자랑 하기 vs 친한 친구 10년 못 보기', 'mbti': 'ENTP'}`

"""
    user_message = f"""
    다음 문장에서 토론 주제와 MBTI를 추출해줘 
    문장 : {user_text}
    """

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {'role': 'system', 'content': system_instruction},
            {'role': 'user', 'content': user_message}
        ],
        response_format={'type': 'json_object'},
        temperature=0.1,
        max_tokens=4096,
        top_p=1
    )
    return json.loads(response.choices[0].message.content)

### 토론 진행 함수

In [16]:
# 토론 진행 함수
def start_debate(topic, mbti):
    client = OpenAI()
    debate_system_instruction = f"""
    넌 MBTI가 {mbti}인 토론가야. 주어진 {mbti}의 성향과 가치관에 완전히 몰입해야해
    지금부터 '{topic}'이라는 주제로 사용자와 토론을 진행할거야
    {mbti}의 관점에서 너의 의견을 논리적이고, 설득력 있으면서 재치있게 주장해

    ## 응답 규칙 ##
    1. 항상 대화의 맥락을 기억하고 이전 대화 내용과 연결되는 답변을 해야 해
    2. 답변은 항상 한국어로, 그리고 2문장 내외로 간결하게 말해줘

    ## 토론 전략 ##
    1. 사용자의 의견이 너의 가치관과 다르다면 명확한 근거를 들어 반박해. 토론을 흥미롭게 만드는 게 너의 역할이야
"""
    chat_history = [{"role": "system", "content": debate_system_instruction}]
    opening_message = f'자 그럼 {topic}을 주제로 {mbti}와의 토론을 시작하겠습니다. 의견을 먼저 말씀해주세요'
    speak(opening_message)

    # 토론 시작
    while True:
        user_opinion = listen()
        if not user_opinion:
            continue
        if user_opinion in ['그만', '종료', '끝내자']:
            speak('토론을 마무리합니다')
            break

        chat_history.append({'role':'user', 'content':user_opinion})

        response = client.chat.completions.create(
            model='gpt-4o',
            messages=chat_history,
            temperature=0.6
        )
        bot_response = response.choices[0].message.content

        chat_history.append({'role':'assistant', 'content':bot_response})
        speak(bot_response)
    

### 실행 함수

In [17]:
# 실행 함수
def main():
    # 1단계 : 봇의 질문
    initial_question = '안녕하세요, 저는 MBTI 토론 파트너입니다! 토론하고 싶은 주제와 상대할 MBTI를 말씀해주세요'
    speak(initial_question)
    while True:
        # 2단계 : 사용자의 답변
        user_input = listen()

        # 3단계 : '주제'와 'MBTI' 추출
        if not user_input:
            continue

        debate_info = extract_debate_info(user_input)

        if debate_info and debate_info.get('topic') != '추출 실패' and debate_info.get('mbti') != '추출 실패':
            topic = debate_info['topic']
            mbti = debate_info['mbti']
            start_debate(topic, mbti)
            break
        else:
            speak('죄송하지만, 주제와 MBTI를 정확히 파악하지 못했어요. 다시 한 번 말씀해주세요')
    pygame.quit()

In [18]:
main()

🤖 : 안녕하세요, 저는 MBTI 토론 파트너입니다! 토론하고 싶은 주제와 상대할 MBTI를 말씀해주세요


KeyboardInterrupt: 

---